This comp's metric is different from others and must be well understood in order to design good bagging schemes. One of it's characteristics is that it **does not depend** on the absolute values of the predicted scores. Only their order is relevant. Hence, to bag serveal models, we just need to sort the scores and use their ranks.

In this work, I'll be proposing an order based bagging scheme that seems to perform very well. You can try it yourself !

In [ ]:
import pandas as pd, numpy as np
import os

In [ ]:
paths = [
    "../input/rfcx-best-performing-public-kernels/kkiller_inference-tpu-rfcx-audio-detection-fast_0861.csv",
    "../input/rfcx-best-performing-public-kernels/submission_khoongweihao_0845.csv",
#     "../input/rfcx-best-performing-public-kernels/submission_mekhdigakhramanian_0824.csv",
]

weights = np.array([0.6, 0.4])
sum(weights)

In [ ]:
df = pd.read_csv(paths[0]).sort_values("recording_id").reset_index(drop=True)
df.head()

In [ ]:
cols = [f"s{i}" for i in range(24)]

In [ ]:
scores = []
for path in paths:
    df = pd.read_csv(path).sort_values("recording_id").reset_index(drop=True)
    score = np.empty((len(df), 24))
    o = df[cols].values.argsort(1)
    score[np.arange(len(df))[:, None], o] = np.arange(24)[None]
    scores.append(score)
scores = np.stack(scores)
scores.shape

In [ ]:
sub_score = np.sum(scores*weights[:, None, None], 0)
print(sub_score.shape)
sub_score

In [ ]:
sub = pd.DataFrame(sub_score, columns=cols)
sub["recording_id"] = df["recording_id"]
sub = sub[["recording_id"] + cols]
print(sub.shape)
sub.head()

In [ ]:
sub.to_csv("submission.csv", index=False)